In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

In [1]:
from fosforio import snowflake
from fosforio import get_dataframe
from datetime import datetime, timedelta

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns

Matplotlib created a temporary cache directory at /tmp/matplotlib-15qcogt4 because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
snowflake.get_connection(connection_name="TTH_REV_OPT_CXN")
data = get_dataframe("BOOKINGS")
data

Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7f45227d1790>
Please close the connection after use!
Reading dataframe from snowflake native connector


,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,EXPECTED_ARRIVAL_DATE,RESERVATION_STATUS,RESERVATION_STATUS_DATE,TOTAL_STAY_NIGHTS,TALLY_DAYS,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,CHILDREN,BABIES,TOTAL_GUESTS,AVG_ROOMS_PER_NIGHT,TOTAL_ROOM_NIGHTS,MEAL,COUNTRY,MARKET_SEGMENT,DISTRIBUTION_CHANNEL,PREVIOUS_CANCELLATIONS,PREVIOUS_BOOKINGS_NOT_CANCELED,RESERVED_ROOM_TYPE,ASSIGNED_ROOM_TYPE,DEPOSIT_TYPE,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR
0,City Hotel,1,272,2021,7,29,16,16-07-2021,Canceled,17-10-2020,2,0.272,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
1,City Hotel,1,307,2021,8,34,20,20-08-2021,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
2,City Hotel,1,258,2021,7,27,2,02-07-2021,Canceled,17-10-2020,2,0.258,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
3,City Hotel,1,307,2021,8,34,20,20-08-2021,Canceled,17-10-2020,2,0.307,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
4,City Hotel,1,279,2021,7,30,23,23-07-2021,Canceled,17-10-2020,2,0.279,0,2,2,0,0,2,1,2,BB,PRT,Groups,TA/TO,1,0,A,A,No Deposit,0,Transient.Party,62.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117100,Resort Hotel,0,269,2023,8,34,24,24-08-2023,Check.Out,10-09-2023,17,17.000,4,13,2,0,0,2,1,17,BB,GBR,Offline TA/TO,TA/TO,0,0,D,D,No Deposit,0,Contract,84.80
117101,Resort Hotel,0,212,2023,8,35,31,31-08-2023,Check.Out,10-09-2023,10,10.000,2,8,2,1,0,3,1,10,BB,GBR,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,89.75
117102,Resort Hotel,0,204,2023,8,35,29,29-08-2023,Check.Out,12-09-2023,14,14.000,4,10,2,0,0,2,1,14,BB,IRL,Direct,Direct,0,0,E,E,No Deposit,0,Transient,153.57
117103,Resort Hotel,0,161,2023,8,35,31,31-08-2023,Check.Out,14-09-2023,14,14.000,4,10,2,0,0,2,1,14,HB,DEU,Offline TA/TO,TA/TO,0,0,A,A,No Deposit,0,Transient,99.06


In [4]:
data.columns

Index(['HOTEL', 'IS_CANCELED', 'LEAD_TIME', 'ARRIVAL_DATE_YEAR', 'MONTH',
       'ARRIVAL_DATE_WEEK_NUMBER', 'ARRIVAL_DATE_DAY_OF_MONTH',
       'EXPECTED_ARRIVAL_DATE', 'RESERVATION_STATUS',
       'RESERVATION_STATUS_DATE', 'TOTAL_STAY_NIGHTS', 'TALLY_DAYS',
       'STAYS_IN_WEEKEND_NIGHTS', 'STAYS_IN_WEEK_NIGHTS', 'ADULTS', 'CHILDREN',
       'BABIES', 'TOTAL_GUESTS', 'AVG_ROOMS_PER_NIGHT', 'TOTAL_ROOM_NIGHTS',
       'MEAL', 'COUNTRY', 'MARKET_SEGMENT', 'DISTRIBUTION_CHANNEL',
       'PREVIOUS_CANCELLATIONS', 'PREVIOUS_BOOKINGS_NOT_CANCELED',
       'RESERVED_ROOM_TYPE', 'ASSIGNED_ROOM_TYPE', 'DEPOSIT_TYPE',
       'DAYS_IN_WAITING_LIST', 'CUSTOMER_TYPE', 'ADR'],
      dtype='object')

In [5]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce').min()

<ipython-input-5-0b66973ecaeb>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce').min()


Timestamp('2021-07-01 00:00:00')

In [6]:
pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce').max()

<ipython-input-6-31e572c4683b>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce').max()


Timestamp('2023-08-31 00:00:00')

In [7]:
data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')

<ipython-input-7-522c35d3d6bc>:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['EXPECTED_ARRIVAL_DATE'] = pd.to_datetime(data['EXPECTED_ARRIVAL_DATE'], errors='coerce')


In [8]:
data.sort_values(by='EXPECTED_ARRIVAL_DATE')['EXPECTED_ARRIVAL_DATE'].tolist()

[Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2021-07-

In [9]:
original_last_date = datetime(2023, 8, 31)
target_date = datetime(2024, 6, 30)

In [10]:
original_min_date = data['EXPECTED_ARRIVAL_DATE'].min()
original_min_date

Timestamp('2021-07-01 00:00:00')

In [11]:
days_to_add = (target_date - original_last_date).days
days_to_add

304

In [12]:
data['EXPECTED_ARRIVAL_DATE'] = data['EXPECTED_ARRIVAL_DATE'] + timedelta(days=days_to_add)

In [13]:
data['EXPECTED_ARRIVAL_DATE'].min()

Timestamp('2022-05-01 00:00:00')

In [14]:
data['EXPECTED_ARRIVAL_DATE'].max()

Timestamp('2024-06-30 00:00:00')

In [15]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

[Row(status='Statement executed successfully.')]

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117105 entries, 0 to 117104
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   HOTEL                           117105 non-null  object        
 1   IS_CANCELED                     117105 non-null  int8          
 2   LEAD_TIME                       117105 non-null  int16         
 3   ARRIVAL_DATE_YEAR               117105 non-null  int16         
 4   MONTH                           117105 non-null  int8          
 5   ARRIVAL_DATE_WEEK_NUMBER        117105 non-null  int8          
 6   ARRIVAL_DATE_DAY_OF_MONTH       117105 non-null  int8          
 7   EXPECTED_ARRIVAL_DATE           117009 non-null  datetime64[ns]
 8   RESERVATION_STATUS              117105 non-null  object        
 9   RESERVATION_STATUS_DATE         117105 non-null  object        
 10  TOTAL_STAY_NIGHTS               117105 non-null  int8   

In [17]:
data["EXPECTED_ARRIVAL_DATE"] = pd.to_datetime(data["EXPECTED_ARRIVAL_DATE"]).dt.strftime("%Y-%m-%d %H:%M:%S")


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117105 entries, 0 to 117104
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   HOTEL                           117105 non-null  object 
 1   IS_CANCELED                     117105 non-null  int8   
 2   LEAD_TIME                       117105 non-null  int16  
 3   ARRIVAL_DATE_YEAR               117105 non-null  int16  
 4   MONTH                           117105 non-null  int8   
 5   ARRIVAL_DATE_WEEK_NUMBER        117105 non-null  int8   
 6   ARRIVAL_DATE_DAY_OF_MONTH       117105 non-null  int8   
 7   EXPECTED_ARRIVAL_DATE           117009 non-null  object 
 8   RESERVATION_STATUS              117105 non-null  object 
 9   RESERVATION_STATUS_DATE         117105 non-null  object 
 10  TOTAL_STAY_NIGHTS               117105 non-null  int8   
 11  TALLY_DAYS                      117105 non-null  float64
 12  STAYS_IN_WEEKEND

In [21]:
df_model=session.createDataFrame(
        data.values.tolist(),
        schema=data.columns.tolist())
df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.BOOKINGS")

TypeError: field EXPECTED_ARRIVAL_DATE: Cannot merge type <class 'snowflake.snowpark.types.StringType'> and <class 'snowflake.snowpark.types.FloatType'>